# Processing And Transforming Features
### Learn how to clean and prepare features for linear regression.

##### Contents:
- Intro to Feature Engineering
- Features without missing values, but aren't in proper format for Linear Regression model
    - Common issues:
        - the column is not numerical (e.g. a zoning code represented using text)  
        - the column is numerical but not ordinal (e.g. zip code values)
        - the column is numerical but isn't representative of the type of relationship with the target column (e.g. year values)
    - Selecting columns without missing values: `train[train.isnull().sum()[train.isnull().sum()==0].index]`
    - Categorical Features
        - pandas.Series.astype('category')
        - pandas.Series.astype('category').cat.codes
        - pandas.Series.astype('category').cat.codes.value_counts()
    - Dummy Coding (creates `n` one-hot columns)
        - dummy_cols = pd.get_dummies()
    - Transforming Improper Numerical Features (e.g. `year`)
- Dealing with missing values
    - Remove rows containing missing values for specific columns 
    - Impute (or replace) missing values using a descriptive statistic from the column
        - Selecting, Imputing
        - pandas.DataFrame.fillna()
    - Pros/Cons of each approach

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('data/AmesHousing.txt', delimiter="\t")
train = data[0:1460]
test = data[1460:]

train_null_counts = train.isnull().sum()
print(train_null_counts)

Order                0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       249
Lot Area             0
Street               0
Alley             1351
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type        11
Mas Vnr Area        11
Exter Qual           0
Exter Cond           0
                  ... 
Bedroom AbvGr        0
Kitchen AbvGr        0
Kitchen Qual         0
TotRms AbvGrd        0
Functional           0
Fireplaces           0
Fireplace Qu       717
Garage Type         74
Garage Yr Blt       75
Garage Finish       75
Garage Cars          0
Garage Area          0
Garage Qual

## 1: Introduction

To understand how linear regression works, we've stuck to using features from the training dataset that contained no missing values and were already in a convenient numeric representation. In this mission, we'll explore how to transform some of the the remaining features so we can use them in our model. Broadly, the process of processing and creating new features is known as [feature engineering](https://en.wikipedia.org/wiki/Feature_engineering). **Feature engineering is a bit of an art and having knowledge in the specific domain (in this case real estate) can help you create better features.** In this mission, we'll focus on some domain-independent strategies that work for all problems.

- In the first half of this mission, we'll focus only on columns that contain no missing values but still aren't in the proper format to use in a linear regression model. 
- In the latter half of this mission, we'll explore some ways to deal with missing values.

Amongst the columns that don't contain missing values, some of the common issues include:

- the column is not numerical (e.g. a zoning code represented using text)
- the column is numerical but not ordinal (e.g. zip code values)
- the column is numerical but isn't representative of the type of relationship with the target column (e.g. year values)

Let's start by filtering the training set to just the columns containing no missing values.

#### Instructions:
- Select just the columns from the train data frame that contain no missing values.
- Assign the resulting data frame, that contains just these columns, to df_no_mv.
- Use the variables display to become familiar with these columns.

In [2]:
df_no_mv = train[train_null_counts[train_null_counts == 0].index]
df_no_mv.head()

Order        PID  MS SubClass MS Zoning  Lot Area Street Lot Shape  \
0      1  526301100           20        RL     31770   Pave       IR1   
1      2  526350040           20        RH     11622   Pave       Reg   
2      3  526351010           20        RL     14267   Pave       IR1   
3      4  526353030           20        RL     11160   Pave       Reg   
4      5  527105010           60        RL     13830   Pave       IR1   

  Land Contour Utilities Lot Config    ...     Enclosed Porch 3Ssn Porch  \
0          Lvl    AllPub     Corner    ...                  0          0   
1          Lvl    AllPub     Inside    ...                  0          0   
2          Lvl    AllPub     Corner    ...                  0          0   
3          Lvl    AllPub     Corner    ...                  0          0   
4          Lvl    AllPub     Inside    ...                  0          0   

  Screen Porch Pool Area Misc Val Mo Sold  Yr Sold  Sale Type  Sale Condition  \
0            0         0        0       5     2010        WD           Normal   
1          120         0        0       6     2010        WD           Normal   
2            0         0    12500       6     2010        WD           Normal   
3            0         0        0       4     2010        WD           Normal   
4            0         0        0       3     2010        WD           Normal   

   SalePrice  
0     215000  
1     105000  
2     172000  
3     244000  
4     189900  

[5 rows x 58 columns]

## 2: Categorical Features

You'll notice that some of the columns in the data frame `df_no_mv` contain string values. If these columns contain only a limited set of uniuqe values, they're known as **categorical features**. As the name suggests, a categorical feature groups a specific training example into a specific *category*. Here are some examples from the dataset:

    >>> train['Utilities'].value_counts()
    AllPub    1457
    NoSewr       2
    NoSeWa       1
    Name: Utilities, dtype: int64

    >>> train['Street'].value_counts()
    Pave    1455
    Grvl       5

    >>> train['House Style'].value_counts()
    1Story    743
    2Story    440
    1.5Fin    160
    SLvl       60
    SFoyer     35
    2.5Unf     11
    1.5Unf      8
    2.5Fin      3
    
To use these features in our model, we need to transform them into numerical representations. Thankfully, pandas makes this easy because the library has a special [categorical data type](https://pandas.pydata.org/pandas-docs/stable/categorical.html). We can convert any column that contains no missing values (or an error will be thrown) to the categorical data type using the `pandas.Series.astype()` method:

    >>> train['Utilities'] = train['Utilities'].astype('categorical')
    
When a column is converted to the categorical data type, pandas assigns a code to each unique value in the column. Unless we access these values directly, most of the pandas manipulation operations that work for string columns will work for categorical ones as well.

We need to use the `.cat` accessor followed by the `.codes` property to actually access the underlying numerical representation of a column:

    >>> train['Utilities'].cat.codes
    
Let's convert all of the text columns that contain no missing values into the categorical data type.

#### Instructions:

- Convert all of the text columns in train to the categorical data type.
- Select the Utilities column, return the categorical codes, and display the unique value counts for those codes: `train['Utilities'].cat.codes.value_counts()`

In [3]:
text_cols = df_no_mv.select_dtypes(include=['object']).columns

for col in text_cols:
    print(col+":", len(train[col].unique()))

MS Zoning: 6
Street: 2
Lot Shape: 4
Land Contour: 4
Utilities: 3
Lot Config: 5
Land Slope: 3
Neighborhood: 26
Condition 1: 9
Condition 2: 6
Bldg Type: 5
House Style: 8
Roof Style: 6
Roof Matl: 5
Exterior 1st: 14
Exterior 2nd: 16
Exter Qual: 4
Exter Cond: 5
Foundation: 6
Heating: 6
Heating QC: 4
Central Air: 2
Electrical: 4
Kitchen Qual: 5
Functional: 7
Paved Drive: 3
Sale Type: 9
Sale Condition: 5


In [4]:
# Example of how this looks (like strings) without accessing values directly
train['Utilities'].astype('category').value_counts()

AllPub    1457
NoSewr       2
NoSeWa       1
Name: Utilities, dtype: int64

In [5]:
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
# pd.options.mode.chained_assignment = None  # default='warn'
train = train.copy() # or can do this to remove warning (because train is a sub-df of `data`)
# Pandas emits a UserWarning to warn you that modifying public does not modify 
# that other DataFrame. If modifying that other DataFrame is not what you intend 
# to do or is not an issue, then you are free to ignore the UserWarning.

for col in text_cols:
    train[col] = train[col].astype('category')

In [6]:
train['Utilities'].cat.codes.value_counts()

0    1457
2       2
1       1
dtype: int64

## 3: Dummy Coding

When we convert a column to the categorical data type, pandas assigns a number from `0` to `n-1` (where `n` is the number of unique values in a column) for each value. The drawback with this approach is that one of the assumptions of linear regression is violated here. Linear regression operates under the assumption that the features are linearly correlated with the target column. For a categorical feature, however, there's no actual numerical meaning to the categorical codes that pandas assigned for that colum. An increase in the `Utilities` column from `1` to `2` has no correlation value with the target column, and the categorical codes are instead used for uniqueness and exclusivity (the category associated with `0` is different than the one associated with `1`).

The common solution is to use a technique called [dummy coding](https://en.wikipedia.org/wiki/Dummy_variable_(statistics)). Instead of having a single column with `n` integer codes, we have `n` binary columns. Here's what that would look like for the `Utilities` column:

| Utilities_AllPub | Utilities_NoSewr | Utilities_NoSeWa |
|------------------|------------------|------------------|
| 1                | 0                | 0                |
| 1                | 0                | 0                |
| 1                | 0                | 0                |
| 1                | 0                | 0                |

Because the original values for the first 4 rows were `AllPub`, in the new scheme, they contain the binary value for true (`1`) in the `Utilities_AllPub` column and `0` for the other 2 columns.

Pandas thankfully has a convenience method to help us apply this transformation for all of the text columns called `pandas.get_dummies()`:

    dummy_cols = pd.get_dummies()

#### Instructions:
- Convert all of the columns in text_cols from the train data frame into dummy columns.
- Delete the original columns from text_cols from the train data frame.

In [7]:
# convert text cols to dummy cols
dummy_cols = pd.get_dummies(train[text_cols])

# drop original text cols
train = train.drop(text_cols, axis=1)

# concat train and dummy cols
train = pd.concat([train, dummy_cols], axis=1)

# check shape
train.shape

(1460, 236)

## 4: Transforming Improper Numerical Features

In the last few screens, we focused on categorical values that were represented as text columns. Some of the numerical columns in the data set are *also* categorical and only have a limited set of unique values. We won't explicitly explore those coumns in this mission, but the feature transformation process is the same if the numbers used in those categories have no numerical meaning.

Let's now look at numerical features that *aren't* categorical, but whose numerical representation needs to be improved. We'll focus on the `Year Remod/Add` and `Year Built` columns:

    >>> train[['Year Remod/Add', 'Year Built']]
    0   1960    1960
    1   1961    1961
    2   1958    1958
    3   1968    1968
    4   1998    1997
    ...
    
The two main issues with these features are:

- Year values aren't representative of how old a house is
- The `Year Remod/Add` column doesn't actually provide useful information for a linear regression model

The challenge with year values like `1960` and `1961` is that they don't do a good capture how old a house is. For example, a house that was built in `1960` but sold in `1980` was sold in half the time one built in `1960` and sold in `2000`. Instead of the years certain events happened, we want the difference between those years. We should create a new column that's the *difference* between both of these columns.    

For this particular piece of information (years until remodeled), this is a sensible approach. Domain knowledge can help you understand how to best transform features to represent information well for a linear model. If you're ever confused about a feature or how it should be represented, reading scientific papers or posts by researchers in the specific domain is critical. Many winners of the [Kaggle data science competition](https://www.dataquest.io/m/239/processing-and-transforming-features/4/kaggle.com), for example, claim that their focus on data preparation and feature engineering combined with common machine learning models helped them win.

#### Instructions:
- Create a new column years_until_remod in the train data frame that represents the difference between Year Remod/Add (the later value) and Year Built (the earlier value).

In [8]:
train['years_until_remod'] = train['Year Remod/Add'] - train['Year Built']

## 5: Missing Values

In the next few screens, we'll focus on handling columns with missing values. When values are missing in a column, there are two main approaches we can take:

- Remove rows containing missing values for specific columns
    - Pro: Rows containing missing values are removed, leaving only clean data for modeling
    - Con: Entire observations from the training set are removed, which can reduce overall prediction accuracy
- Impute (or replace) missing values using a descriptive statistic from the column
    - Pro: Missing values are replaced with potentially similar estimates, preserving the rest of the observation in the model.
    - Con: Depending on the approach, we may be adding noisy data for the model to learn
    
Given that we only have 1460 training examples (with ~80 potentially useful features), we don't want to remove any of these rows from the dataset. Let's instead focus on **imputation** techniques.

We'll focus on columns that contain at least 1 missing value but less than 365 missing values (or 25% of the number of rows in the training set). There's no strict threshold, and **many people instead use a 50% cutoff** (if half the values in a column are missing, it's automatically dropped). Having some domain knowledge can help with determining an acceptable cutoff value.

#### Instructions:

- Select only the columns from train that contain more than 0 missing values but less than 584 missing values. Assign the resulting data frame to df_missing_values.
- Display the number of missing values for each column in df_missing_values.
- Display the data type for each column in df_missing_values.    

In [9]:
train_null_counts = train.isnull().sum()
train_null_counts_subset = train_null_counts[(train_null_counts > 0) & (train_null_counts < 584)]
train_null_counts_subset.index

df_missing_values = train[train_null_counts_subset.index]
df_missing_values.head()

Lot Frontage Mas Vnr Type  Mas Vnr Area Bsmt Qual Bsmt Cond Bsmt Exposure  \
0         141.0        Stone         112.0        TA        Gd            Gd   
1          80.0         None           0.0        TA        TA            No   
2          81.0      BrkFace         108.0        TA        TA            No   
3          93.0         None           0.0        TA        TA            No   
4          74.0         None           0.0        Gd        TA            No   

  BsmtFin Type 1  BsmtFin SF 1 BsmtFin Type 2  BsmtFin SF 2  Bsmt Unf SF  \
0            BLQ         639.0            Unf           0.0        441.0   
1            Rec         468.0            LwQ         144.0        270.0   
2            ALQ         923.0            Unf           0.0        406.0   
3            ALQ        1065.0            Unf           0.0       1045.0   
4            GLQ         791.0            Unf           0.0        137.0   

   Total Bsmt SF  Bsmt Full Bath  Bsmt Half Bath Garage Type  Garage Yr Blt  \
0         1080.0             1.0             0.0      Attchd         1960.0   
1          882.0             0.0             0.0      Attchd         1961.0   
2         1329.0             0.0             0.0      Attchd         1958.0   
3         2110.0             1.0             0.0      Attchd         1968.0   
4          928.0             0.0             0.0      Attchd         1997.0   

  Garage Finish Garage Qual Garage Cond  
0           Fin          TA          TA  
1           Unf          TA          TA  
2           Unf          TA          TA  
3           Fin          TA          TA  
4           Fin          TA          TA

## 6: Imputing Missing Values

It looks like about half of the columns in `df_missing_values` are string columns (`object` data type), while about half are `float64` columns. For numerical columns with missing values, a common strategy is to compute the mean, median, or mode of each column and replace all missing values in that column with that value.

Because imputation is a common task, pandas contains a method named `pandas.DataFrame.fillna()` that we can use for this. If we pass in a value, all of the missing values (`NaN`) in the data frame are replaced by that value.

#### Instructions:
- Impute the missing values from float_cols with the column's mean.
- Check for any missing values in either string_cols or float_cols.

In [10]:
print(df_missing_values.shape[1]) # total features count
print(df_missing_values.select_dtypes(['float']).shape[1]) # numerical features count
print(df_missing_values.select_dtypes(['object']).shape[1]) # string features count

19
9
10


In [11]:
# select floats df
float_cols = df_missing_values.select_dtypes(include=['float'])
# prevent warning
df_missing_values = df_missing_values.copy()
# fill float NaNs with mean of columns (can also use np.mean)
df_missing_values[float_cols.columns] = df_missing_values[float_cols.columns].fillna(df_missing_values.mean())
# check float NaN counts
df_missing_values[float_cols.columns].isnull().sum()

Lot Frontage      0
Mas Vnr Area      0
BsmtFin SF 1      0
BsmtFin SF 2      0
Bsmt Unf SF       0
Total Bsmt SF     0
Bsmt Full Bath    0
Bsmt Half Bath    0
Garage Yr Blt     0
dtype: int64

## 7: Next Steps

In this mission, we explored a few different techniques for transforming features into appropriate representations for a linear regression model. Next in this course is a guided project, where you'll practice the techniques you've learned in this course to build better linear regression models.